# 9 データベースとSQLの基礎 (Python)

- **[9.7 Pythonとの連携](#9.7-Pythonとの連携)**
    - [9.7.1 Pythonからの接続方法と具体例](#9.7.1-Pythonからの接続方法と具体例)
<br><br>
- **[9.8 総合問題](#9.8-総合問題)** 
    - [9.8.3 総合問題3](#9.8.3-総合問題3)

***

## 9.7 Pythonとの連携
この章のゴール：PythonとMysqlを接続して、データのハンドリングができる

### 9.7.1 Pythonからの接続方法と具体例 
キーワード：pymysql

この章では、前章までで学んだSQLとPythonを連携して、データをハンドリングする手法を身につけましょう。基本的には、Pythonからの接続方法さえ覚えれば、あとはSQLの問題になりますので、前章のSQLができれば大丈夫です。

なお、今回はモジュールを使いますので、以下のコマンドをterminal上で実行して、ダウンロードしてください。(新しくモジュールをダウンロードする時によく使いますので、余力がある方は使えるようになりましょう。)　terminalはjupyterのトップ画面へいき、右側にある「New」からterminalを選択して実行します。黒い画面にコマンド打ち込んでいきます。

pip install pymysql

以下から実装です。なお、データをダウンロードした時に使用したパラメータを見て、以下のconnectメソッドのパラメータも適宜変更して実行してください。

In [1]:
import pymysql.cursors

In [2]:
dbh = pymysql.connect(
    host='zdb',
    user='root',
    password='gci',
    db='TEST1',
    charset='utf8',
    cursorclass=pymysql.cursors.DictCursor )#カーソルのクラスを指定できます。ここでは辞書型にしています。

#カーソル取得
stmt = dbh.cursor()

#処理したいSQL
sql = "select * from meibo limit 10"

#SQL実行
stmt.execute(sql)

#結果取得 辞書型で取得できます。
rows = stmt.fetchall()

#表示
for row in rows:
    print(row['id'],row['name'],row['age'],row['class'],row['height'])


#切る
stmt.close()
dbh.close()

1 Yamada 14 1 150
2 Tanaka 13 2 150
3 Suzuki 13 1 150
4 Kato 15 2 150
5 Ito 12 3 155
6 Takeuchi 16 2 155
7 Kimura 11 3 160
8 Sato 14 1 160
100 dummy_Yamada None None None
200 dummy_Tanaka None None None


基本的には、SQLでの操作になりますので、前に学んだ記述方法と同じです。

contextlibにあるclosingとwith文を組み合わせると，以下のようにしてcursorを自動でcloseすることができます。withを使うと例外が発生したときでもcursorを自動的に閉じてくれます。

In [3]:
from contextlib import closing

In [4]:
db = pymysql.connect(
    host='zdb',
    user='root',
    password='gci',
    db='TEST1',
    charset='utf8',
    cursorclass=pymysql.cursors.DictCursor )

In [5]:
with closing(db.cursor()) as cur:
    cur.execute("select * from meibo limit 10")
    row = cur.fetchone()
    while row is not None:
        print(row['id'],row['name'],row['age'],row['class'],row['height'])
        row = cur.fetchone()

1 Yamada 14 1 150
2 Tanaka 13 2 150
3 Suzuki 13 1 150
4 Kato 15 2 150
5 Ito 12 3 155
6 Takeuchi 16 2 155
7 Kimura 11 3 160
8 Sato 14 1 160
100 dummy_Yamada None None None
200 dummy_Tanaka None None None


以下のようにパラメータを設定して、データを抽出することもできます。以下はageをパラメータとして、15歳以上のデータを抽出しています。

In [6]:
with closing(db.cursor()) as cur:
    age = 15
    cur.execute("select * from meibo where  age>= %s", (age,))
    row = cur.fetchone()
    while row is not None:
        print(row['name'],row['age'])
        row = cur.fetchone()

Kato 15
Takeuchi 16


In [7]:
db.close()

***

## 9.8 総合問題

### 9.8.3 総合問題3

9.8.2と同じデータベースを使って以下の問に答えてください。

(1)給料支払日毎の支払い済み給料の四分位点、最頻値を求めてください。

4分位点はnumpyのpercentile関数、最頻値はstatisticsのmode関数で求められます。いずれもmysql上で計算するのは不可能ではありませんが、python上で計算してみてください。

In [8]:
# 解答例
import pymysql.cursors
import numpy as np
import statistics as st
from contextlib import closing

dbh = pymysql.connect(
    host='zdb',
    user='root',
    password='gci',
    db='foodmart',
    charset='utf8',
    cursorclass=pymysql.cursors.Cursor )

sqls = [];
dates = [];

#日付毎のデータを取得するSQLを生成する。※とりあえず全レコード取得してから分割しても良い。
with closing(dbh.cursor()) as cur:
    cur.execute("select date_format(t.pay_date,'%Y-%m-%d') date from salary t group by t.pay_date")
    row = cur.fetchone()
    while row is not None:
        sqls.append("select salary_paid from salary where pay_date = '" + row[0] +"';")
        dates.append(row[0])
        row = cur.fetchone()

print("日付\t25パーセンタイル、\t50パーセンタイル\t75パーセンタイル\t最頻値")
with closing(dbh.cursor()) as cur:
    for sql in sqls:
        cur.execute(sql)
        result = cur.fetchall()
        array = np.array(result).astype(np.float64)
        print(dates.pop(0),"\t",
              np.percentile(array ,25),"\t",
              np.percentile(array ,50),"\t",
              np.percentile(array ,75),"\t",
              st.mode(result)[0])
            
            
dbh.close()


日付	25パーセンタイル、	50パーセンタイル	75パーセンタイル	最頻値
1997-01-01 	 3.375 	 3.96 	 6.1425 	 6.3000
1997-02-01 	 3.360925 	 3.96 	 6.1425 	 6.3000
1997-03-01 	 3.35815 	 3.96 	 6.1425 	 6.3000
1997-04-01 	 3.3542 	 3.96 	 6.1425 	 6.3000
1997-05-01 	 3.372025 	 3.96 	 6.1425 	 6.3000
1997-06-01 	 3.363425 	 3.96 	 6.1425 	 6.3000
1997-07-01 	 3.356725 	 3.96 	 6.1425 	 6.3000
1997-08-01 	 3.34105 	 3.96 	 6.1425 	 6.3000
1997-09-01 	 3.349725 	 3.96 	 6.1425 	 6.3000
1997-10-01 	 3.368775 	 3.96 	 6.1425 	 6.3000
1997-11-01 	 3.358575 	 3.96 	 6.1425 	 6.3000
1997-12-01 	 3.35905 	 3.96 	 6.1425 	 6.3000
1998-01-01 	 3.373 	 4.5 	 9.5714 	 7.0000
1998-02-01 	 3.3716 	 4.5 	 9.5714 	 7.0000
1998-03-01 	 3.37345 	 4.5 	 9.5714 	 7.0000
1998-04-01 	 3.375 	 4.5 	 9.5714 	 7.0000
1998-05-01 	 3.38615 	 4.5 	 9.5714 	 7.0000
1998-06-01 	 3.36025 	 4.5 	 9.5714 	 7.0000
1998-07-01 	 3.3737 	 4.5 	 9.5714 	 7.0000
1998-08-01 	 3.37725 	 4.5 	 9.5714 	 7.0000
1998-09-01 	 3.36775 	 4.5 	 9.5714 	 7.0000
1998-10